## IMPORTS



In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## EJ 9) - Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente información 

###(patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha)

##donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto. Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.


In [9]:
patentamientos = [(0,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-01-27'),
 (1, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (2, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Rio Negro', '2017-04-04'),
 (3, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Santa Fe', '2017-02-04'),
 (4, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Cordoba', '2017-04-28'),
 (5, 'Marca 2', 'Modelo 3', 'Version 1', 'Auto', 'Santa Cruz', '2017-03-14'),
 (6, 'Marca 1', 'Modelo 2', 'Version 2', 'Moto', 'Cordoba', '2017-02-02'),
 (7,
  'Marca 3',
  'Modelo 1',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-02-18'),
 (8, 'Marca 3', 'Modelo 2', 'Version 2', 'Moto', 'Buenos Aires', '2017-04-29'),
 (9, 'Marca 3', 'Modelo 2', 'Version 2', 'Camion', 'Rio Negro', '2017-04-30'),
 (10, 'Marca 1', 'Modelo 2', 'Version 1', 'Moto', 'Santa Cruz', '2017-01-18'),
 (11, 'Marca 3', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-03-16'),
 (12,
  'Marca 1',
  'Modelo 2',
  'Version 3',
  'Moto',
  'Buenos Aires',
  '2017-03-26'),
 (13, 'Marca 2', 'Modelo 2', 'Version 3', 'Auto', 'Santa Cruz', '2017-04-05'),
 (14,
  'Marca 1',
  'Modelo 3',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-04-27'),
 (15, 'Marca 2', 'Modelo 2', 'Version 2', 'Moto', 'Neuquen', '2017-01-28'),
 (16, 'Marca 3', 'Modelo 1', 'Version 3', 'Pickup', 'Cordoba', '2017-03-31'),
 (17, 'Marca 2', 'Modelo 2', 'Version 1', 'Auto', 'Cordoba', '2017-02-09'),
 (18, 'Marca 3', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-04-22'),
 (19, 'Marca 2', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-04-25'),
 (20,
  'Marca 2',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-02-10'),
 (21, 'Marca 3', 'Modelo 2', 'Version 3', 'Moto', 'Rio Negro', '2017-04-29'),
 (22, 'Marca 2', 'Modelo 3', 'Version 3', 'Camion', 'Cordoba', '2017-02-24'),
 (23, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Cordoba', '2017-02-28'),
 (24,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Auto',
  'Buenos Aires',
  '2017-01-03'),
 (25, 'Marca 1', 'Modelo 3', 'Version 2', 'Camion', 'Santa Fe', '2017-02-19'),
 (26, 'Marca 1', 'Modelo 1', 'Version 2', 'Moto', 'Neuquen', '2017-02-26'),
 (27, 'Marca 1', 'Modelo 3', 'Version 1', 'Camion', 'Neuquen', '2017-03-19'),
 (28, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-04-11'),
 (29, 'Marca 1', 'Modelo 3', 'Version 1', 'Pickup', 'Cordoba', '2017-01-17'),
 (30, 'Marca 3', 'Modelo 2', 'Version 1', 'Pickup', 'Cordoba', '2017-04-22'),
 (31,
  'Marca 3',
  'Modelo 3',
  'Version 2',
  'Pickup',
  'Santa Cruz',
  '2017-04-23'),
 (32, 'Marca 1', 'Modelo 3', 'Version 3', 'Auto', 'Santa Cruz', '2017-04-17'),
 (33,
  'Marca 1',
  'Modelo 2',
  'Version 2',
  'Auto',
  'Buenos Aires',
  '2017-01-24'),
 (34, 'Marca 1', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-01-06'),
 (35, 'Marca 2', 'Modelo 1', 'Version 3', 'Pickup', 'Cordoba', '2017-04-18'),
 (36, 'Marca 2', 'Modelo 1', 'Version 3', 'Moto', 'Rio Negro', '2017-03-02'),
 (37, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Cordoba', '2017-03-30'),
 (38, 'Marca 1', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-03-16'),
 (39, 'Marca 2', 'Modelo 1', 'Version 3', 'Auto', 'Cordoba', '2017-03-20'),
 (40, 'Marca 3', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-04-16'),
 (41,
  'Marca 2',
  'Modelo 3',
  'Version 1',
  'Moto',
  'Buenos Aires',
  '2017-01-29'),
 (42, 'Marca 2', 'Modelo 2', 'Version 3', 'Moto', 'Neuquen', '2017-02-26'),
 (43,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Santa Cruz',
  '2017-04-14'),
 (44,
  'Marca 2',
  'Modelo 3',
  'Version 3',
  'Moto',
  'Buenos Aires',
  '2017-03-29'),
 (45, 'Marca 3', 'Modelo 1', 'Version 1', 'Camion', 'Neuquen', '2017-03-07'),
 (46, 'Marca 2', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-02-04'),
 (47, 'Marca 2', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-02-21'),
 (48, 'Marca 2', 'Modelo 3', 'Version 1', 'Pickup', 'Cordoba', '2017-02-07'),
 (49,
  'Marca 2',
  'Modelo 2',
  'Version 1',
  'Pickup',
  'Santa Cruz',
  '2017-04-07'),
 (50, 'Marca 2', 'Modelo 2', 'Version 2', 'Moto', 'Rio Negro', '2017-01-12'),
 (51, 'Marca 1', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-01-30'),
 (52, 'Marca 1', 'Modelo 2', 'Version 1', 'Camion', 'Rio Negro', '2017-01-23'),
 (53, 'Marca 1', 'Modelo 1', 'Version 3', 'Moto', 'Santa Fe', '2017-02-14'),
 (54, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Rio Negro', '2017-03-19'),
 (55,
  'Marca 3',
  'Modelo 1',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-01-22'),
 (56,
  'Marca 1',
  'Modelo 2',
  'Version 1',
  'Auto',
  'Buenos Aires',
  '2017-01-23'),
 (57,
  'Marca 2',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-03-19'),
 (58, 'Marca 2', 'Modelo 2', 'Version 3', 'Moto', 'Cordoba', '2017-01-01'),
 (59, 'Marca 3', 'Modelo 2', 'Version 2', 'Auto', 'Neuquen', '2017-04-09'),
 (60, 'Marca 1', 'Modelo 1', 'Version 1', 'Moto', 'Santa Cruz', '2017-04-17'),
 (61, 'Marca 2', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-02-15'),
 (62,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-03-20'),
 (63, 'Marca 3', 'Modelo 1', 'Version 2', 'Moto', 'Santa Cruz', '2017-03-22'),
 (64, 'Marca 2', 'Modelo 3', 'Version 1', 'Auto', 'Santa Cruz', '2017-01-06'),
 (65,
  'Marca 1',
  'Modelo 3',
  'Version 1',
  'Pickup',
  'Santa Cruz',
  '2017-01-31'),
 (66, 'Marca 1', 'Modelo 2', 'Version 1', 'Pickup', 'Cordoba', '2017-03-01'),
 (67,
  'Marca 3',
  'Modelo 1',
  'Version 2',
  'Pickup',
  'Buenos Aires',
  '2017-04-12'),
 (68, 'Marca 1', 'Modelo 1', 'Version 2', 'Moto', 'Neuquen', '2017-01-22'),
 (69, 'Marca 1', 'Modelo 2', 'Version 3', 'Moto', 'Rio Negro', '2017-01-07'),
 (70, 'Marca 1', 'Modelo 2', 'Version 3', 'Auto', 'Santa Fe', '2017-03-05'),
 (71, 'Marca 1', 'Modelo 1', 'Version 3', 'Camion', 'Cordoba', '2017-03-03'),
 (72, 'Marca 2', 'Modelo 2', 'Version 1', 'Camion', 'Cordoba', '2017-02-20'),
 (73,
  'Marca 2',
  'Modelo 2',
  'Version 2',
  'Moto',
  'Buenos Aires',
  '2017-03-20'),
 (74, 'Marca 1', 'Modelo 1', 'Version 3', 'Moto', 'Rio Negro', '2017-01-29'),
 (75, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-01-10'),
 (76, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Cordoba', '2017-01-03'),
 (77, 'Marca 3', 'Modelo 2', 'Version 2', 'Auto', 'Neuquen', '2017-01-03'),
 (78,
  'Marca 1',
  'Modelo 1',
  'Version 2',
  'Camion',
  'Buenos Aires',
  '2017-02-18'),
 (79, 'Marca 3', 'Modelo 2', 'Version 1', 'Pickup', 'Rio Negro', '2017-02-15'),
 (80, 'Marca 2', 'Modelo 1', 'Version 1', 'Auto', 'Santa Fe', '2017-04-02'),
 (81, 'Marca 1', 'Modelo 1', 'Version 1', 'Pickup', 'Neuquen', '2017-02-12'),
 (82, 'Marca 3', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-03-19'),
 (83, 'Marca 3', 'Modelo 2', 'Version 1', 'Camion', 'Santa Fe', '2017-01-15'),
 (84, 'Marca 3', 'Modelo 2', 'Version 1', 'Auto', 'Cordoba', '2017-03-12'),
 (85,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-03-26'),
 (86, 'Marca 3', 'Modelo 2', 'Version 3', 'Moto', 'Santa Fe', '2017-02-19'),
 (87, 'Marca 3', 'Modelo 3', 'Version 3', 'Camion', 'Rio Negro', '2017-01-08'),
 (88, 'Marca 2', 'Modelo 2', 'Version 1', 'Auto', 'Rio Negro', '2017-04-19'),
 (89, 'Marca 3', 'Modelo 3', 'Version 1', 'Auto', 'Santa Fe', '2017-03-03'),
 (90, 'Marca 2', 'Modelo 1', 'Version 3', 'Auto', 'Cordoba', '2017-01-15'),
 (91,
  'Marca 1',
  'Modelo 1',
  'Version 2',
  'Moto',
  'Buenos Aires',
  '2017-03-30'),
 (92, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Rio Negro', '2017-02-03'),
 (93, 'Marca 2', 'Modelo 2', 'Version 1', 'Pickup', 'Santa Fe', '2017-02-23'),
 (94,
  'Marca 2',
  'Modelo 2',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-03-29'),
 (95, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (96,
  'Marca 1',
  'Modelo 1',
  'Version 3',
  'Pickup',
  'Buenos Aires',
  '2017-03-15'),
 (97, 'Marca 2', 'Modelo 1', 'Version 1', 'Camion', 'Santa Fe', '2017-04-28'),
 (98, 'Marca 3', 'Modelo 2', 'Version 1', 'Moto', 'Neuquen', '2017-04-03'),
 (99,
  'Marca 2',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Santa Cruz',
  '2017-03-03')]

patentamientosRDD = sc.parallelize(patentamientos, 8)
patentamientosRDD.take(5)

[(0,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-01-27'),
 (1, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (2, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Rio Negro', '2017-04-04'),
 (3, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Santa Fe', '2017-02-04'),
 (4, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Cordoba', '2017-04-28')]

###(patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha)

la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.

In [52]:
#Filtro por provincia de Buenos Aires
Patentamientos = patentamientosRDD.filter(lambda x: x[5] == 'Buenos Aires')

In [53]:
#Filtro por Mes y Año
#Patentamientos = Patentamientos.filter(lambda x: x[6].startswith('2017-04'))

In [54]:
#Reorganizo el RDD para poder realizar el reduceByKey ( (tipo_vehiculo, marca, modelo), 1  )
Patentamientos = Patentamientos.map(lambda x: ((x[4], x[1], x[2]),1) )

In [55]:
Patentamientos.collect()


[(('Camion', 'Marca 3', 'Modelo 1'), 1),
 (('Camion', 'Marca 3', 'Modelo 1'), 1),
 (('Moto', 'Marca 3', 'Modelo 2'), 1),
 (('Moto', 'Marca 1', 'Modelo 2'), 1),
 (('Auto', 'Marca 1', 'Modelo 3'), 1),
 (('Auto', 'Marca 1', 'Modelo 2'), 1),
 (('Moto', 'Marca 2', 'Modelo 3'), 1),
 (('Moto', 'Marca 2', 'Modelo 3'), 1),
 (('Camion', 'Marca 3', 'Modelo 1'), 1),
 (('Auto', 'Marca 1', 'Modelo 2'), 1),
 (('Camion', 'Marca 1', 'Modelo 3'), 1),
 (('Pickup', 'Marca 3', 'Modelo 1'), 1),
 (('Moto', 'Marca 2', 'Modelo 2'), 1),
 (('Camion', 'Marca 1', 'Modelo 1'), 1),
 (('Camion', 'Marca 3', 'Modelo 1'), 1),
 (('Moto', 'Marca 1', 'Modelo 1'), 1),
 (('Camion', 'Marca 2', 'Modelo 2'), 1),
 (('Pickup', 'Marca 1', 'Modelo 1'), 1)]

In [56]:
#Sumo las cantidades de patentamientos ( (tipo_vehiculo, marca, modelo), cant_totales_patentamientos  )
Patentamientos = Patentamientos.reduceByKey(lambda a,b: a+b)

In [57]:
#Reorganizo mi RDD de la forma ( tipo_vehiculo, ( (marca, modelo) , cant_totales_patentamientos ) ) 
Patentamientos = Patentamientos.map(lambda x: ( x[0][0],( (x[0][1], x[0][2]) , x[1] ) ) )

In [58]:
Patentamientos.collect()

[('Moto', (('Marca 2', 'Modelo 2'), 1)),
 ('Camion', (('Marca 2', 'Modelo 2'), 1)),
 ('Camion', (('Marca 3', 'Modelo 1'), 4)),
 ('Auto', (('Marca 1', 'Modelo 3'), 1)),
 ('Camion', (('Marca 1', 'Modelo 3'), 1)),
 ('Camion', (('Marca 1', 'Modelo 1'), 1)),
 ('Moto', (('Marca 1', 'Modelo 1'), 1)),
 ('Moto', (('Marca 2', 'Modelo 3'), 2)),
 ('Moto', (('Marca 3', 'Modelo 2'), 1)),
 ('Moto', (('Marca 1', 'Modelo 2'), 1)),
 ('Auto', (('Marca 1', 'Modelo 2'), 2)),
 ('Pickup', (('Marca 3', 'Modelo 1'), 1)),
 ('Pickup', (('Marca 1', 'Modelo 1'), 1))]

In [59]:
#Me quedo con el mas patentado. 
LosMasPatentados = Patentamientos.reduceByKey(lambda a,b: a if a[1] > b[1] else b )

In [60]:
LosMasPatentados.collect()

[('Moto', (('Marca 2', 'Modelo 3'), 2)),
 ('Camion', (('Marca 3', 'Modelo 1'), 4)),
 ('Auto', (('Marca 1', 'Modelo 2'), 2)),
 ('Pickup', (('Marca 1', 'Modelo 1'), 1))]

In [61]:
#Resultado Final 
LosMasPatentados = LosMasPatentados.map(lambda x: (x[0], x[1][0][0], x[1][0][1]))

In [62]:
LosMasPatentados.collect()

[('Moto', 'Marca 2', 'Modelo 3'),
 ('Camion', 'Marca 3', 'Modelo 1'),
 ('Auto', 'Marca 1', 'Modelo 2'),
 ('Pickup', 'Marca 1', 'Modelo 1')]

## 10- A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: 

#####(id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias)

## Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato
##### (id_videojuego, titulo, rating_pegi, rating_esbr). 

## Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta información escribir un programa en pySpark que permita: 


In [ ]:
InfoJuego = [(4, 'Mario Kart 8 Deluxe', 66, 21),
 (3, "Luigi's Mansion", 53, 94),
 (1, 'Kirby', 26, 84),
 (1, 'Kirby', 16, 6),
 (6, 'Super Mario Bros', 56, 19),
 (5, 'The Legend of Zelda', 97, 25),
 (2, 'Super Mario Maker 2', 51, 88),
 (6, 'Super Mario Bros', 100, 13),
 (4, 'Mario Kart 8 Deluxe', 59, 12),
 (2, 'Super Mario Maker 2', 56, 89),
 (4, 'Mario Kart 8 Deluxe', 33, 41),
 (3, "Luigi's Mansion", 24, 77),
 (7, 'Astral Chain', 27, 62),
 (4, 'Mario Kart 8 Deluxe', 84, 44),
 (4, 'Mario Kart 8 Deluxe', 74, 1),
 (5, 'The Legend of Zelda', 71, 100),
 (7, 'Astral Chain', 95, 56),
 (6, 'Super Mario Bros', 16, 84),
 (3, "Luigi's Mansion", 54, 91),
 (7, 'Astral Chain', 34, 11),
 (4, 'Mario Kart 8 Deluxe', 76, 15),
 (2, 'Super Mario Maker 2', 9, 91),
 (6, 'Super Mario Bros', 59, 1),
 (3, "Luigi's Mansion", 56, 73),
 (1, 'Kirby', 41, 13),
 (2, 'Super Mario Maker 2', 70, 33),
 (4, 'Mario Kart 8 Deluxe', 10, 73),
 (7, 'Astral Chain', 77, 32),
 (3, "Luigi's Mansion", 4, 22),
 (1, 'Kirby', 63, 36),
 (1, 'Kirby', 53, 78),
 (3, "Luigi's Mansion", 35, 63),
 (6, 'Super Mario Bros', 54, 84),
 (6, 'Super Mario Bros', 76, 94),
 (4, 'Mario Kart 8 Deluxe', 7, 30),
 (2, 'Super Mario Maker 2', 57, 26),
 (7, 'Astral Chain', 35, 30),
 (3, "Luigi's Mansion", 79, 93),
 (4, 'Mario Kart 8 Deluxe', 27, 6),
 (6, 'Super Mario Bros', 14, 3),
 (7, 'Astral Chain', 73, 57),
 (6, 'Super Mario Bros', 56, 28),
 (5, 'The Legend of Zelda', 16, 26),
 (1, 'Kirby', 56, 35),
 (2, 'Super Mario Maker 2', 70, 86),
 (1, 'Kirby', 46, 99),
 (7, 'Astral Chain', 98, 63),
 (2, 'Super Mario Maker 2', 78, 43),
 (1, 'Kirby', 88, 81),
 (6, 'Super Mario Bros', 76, 80)]

In [ ]:
VentaJuegos = [
  (4, 3, '2019-08-19', 435, 148),
 (3, 1, '2019-12-12', 313, 88),
 (1, 9, '2019-09-19', 394, 322),
 (1, 9, '2019-07-31', 181, 201),
 (6, 6, '2019-10-13', 353, 194),
 (5, 8, '2019-09-21', 131, 155),
 (2, 2, '2019-03-15', 390, 9),
 (6, 8, '2019-12-11', 146, 11),
 (4, 1, '2019-08-12', 178, 151),
 (2, 1, '2019-01-22', 349, 345),
 (4, 5, '2019-05-27', 291, 274),
 (3, 3, '2019-08-05', 261, 262),
 (7, 3, '2019-04-26', 454, 82),
 (4, 1, '2019-08-24', 84, 94),
 (4, 9, '2019-06-05', 160, 84),
 (5, 4, '2019-08-16', 252, 24),
 (7, 3, '2019-09-16', 30, 149),
 (6, 10, '2019-11-16', 441, 309),
 (3, 6, '2019-11-17', 104, 260),
 (7, 3, '2019-04-30', 371, 343),
 (4, 3, '2019-04-13', 33, 29),
 (2, 6, '2019-04-11', 67, 167),
 (6, 5, '2019-08-21', 246, 308),
 (3, 5, '2019-07-23', 264, 98),
 (1, 1, '2019-01-28', 173, 337),
 (2, 3, '2019-10-20', 55, 284),
 (4, 2, '2019-10-05', 22, 10),
 (7, 9, '2019-01-02', 59, 157),
 (3, 9, '2019-07-08', 136, 299),
 (1, 9, '2019-03-25', 408, 303),
 (1, 6, '2019-04-14', 195, 15),
 (3, 10, '2019-08-26', 370, 313),
 (6, 2, '2019-04-25', 274, 203),
 (6, 2, '2019-02-06', 408, 64),
 (4, 7, '2019-11-13', 169, 209),
 (2, 4, '2019-01-07', 391, 318),
 (7, 6, '2019-08-17', 434, 271),
 (3, 10, '2019-05-25', 351, 262),
 (4, 5, '2019-01-12', 445, 168),
 (6, 10, '2019-11-11', 9, 314),
 (7, 7, '2019-11-13', 406, 316),
 (6, 6, '2019-12-04', 199, 191),
 (5, 1, '2019-04-23', 35, 16),
 (1, 10, '2019-08-22', 242, 321),
 (2, 6, '2019-07-11', 174, 350),
 (1, 7, '2019-11-03', 406, 26),
 (7, 2, '2019-06-05', 287, 282),
 (2, 2, '2019-07-29', 447, 194),
 (1, 3, '2019-09-23', 231, 145),
 (6, 7, '2019-08-22', 452, 252)]

## A) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

In [ ]:
ventasJuegosRDD = sc.parallelize(VentaJuegos)
infoJuegosRDD = sc.parallelize(InfoJuego)

##### (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias) 

##### (id_videojuego, titulo, rating_pegi, rating_esbr).



In [ ]:
Ventas = ventasJuegosRDD.map(lambda x: ( x[0], x[4]) ).reduceByKey(lambda a,b: a+b)
Ventas.collect()

[(4, 1167), (6, 1846), (2, 1667), (3, 1582), (1, 1670), (5, 195), (7, 1600)]

In [ ]:
Informacion = infoJuegosRDD.map(lambda x: (x[0],x[1]))

In [ ]:
JuntoDatos = Ventas.leftOuterJoin(Informacion).distinct()
JuntoDatos.collect()


[(6, (1846, 'Super Mario Bros')),
 (5, (195, 'The Legend of Zelda')),
 (2, (1667, 'Super Mario Maker 2')),
 (3, (1582, "Luigi's Mansion")),
 (1, (1670, 'Kirby')),
 (4, (1167, 'Mario Kart 8 Deluxe')),
 (7, (1600, 'Astral Chain'))]

In [ ]:

Resultado = JuntoDatos.map(lambda x: (x[0],x[1][1], x[1][0])).reduce(lambda a,b: a if a[2] > b[2] else b)
ResultadoRDD = sc.parallelize(Resultado, 8)
ResultadoRDD.collect()

NameError: ignored

ANOTAR ESTO !!!!!!!!!!!! TIPICO ERROR

## B) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) 


##### (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias) 

##### (id_videojuego, titulo, rating_pegi, rating_esbr).


In [ ]:
MasVendidos = JuntoDatos.map(lambda x: (x[0],x[1][1], x[1][0]))
MasVendidos.collect()

In [ ]:
CalcularTasa = ventasJuegosRDD.map(lambda x: (x[0], x[1], x[4]/x[3]))
CalcularTasa.collect()


In [ ]:
ID_MasVendido = Resultado[0]
FiltroJuego = CalcularTasa.filter(lambda x: x[0] == ID_MasVendido)
FiltroJuego.reduce(lambda a,b : a if a[2]>b[2] else b )